In [ ]:
# test
# pull out all of the headlines from the New York Times homepage
from selenium import webdriver
ecxeutable_path='D:/app/drivers/chromedriver.exe'
driver = webdriver.Chrome()
driver.get("https://www.nytimes.com")
headlines = driver.find_elements_by_class_name("story-heading")
for headline in headlines:
    print(headline.text.strip())

In [46]:
####
#Author: brandon chiazza and Jiahao Li
#version 2.0
#references: 
#https://www.programiz.com/python-programming/working-csv-files
#https://boto3.amazonaws.com/v1/documentation/api/latest/reference/services/s3.html#S3.Client.create_bucket
#https://realpython.com/python-boto3-aws-s3/
#CLI aws s3api create-bucket --bucket my-bucket-name --region us-west-2 --create-bucket-configuration LocationConstraint=us-west-2
#https://robertorocha.info/setting-up-a-selenium-web-scraper-on-aws-lambda-with-python/ 
##

###Load libraries
import awscli
import selenium
import boto3
import pandas as pd
import time

from selenium import webdriver


####SCRAPE THE WEBSITE######
###call the webdriver
ecxeutable_path='D:/app/drivers/chromedriver.exe'
browser = webdriver.Chrome(ecxeutable_path)

#enter the url path that needs to be accessed by webdriver
browser.get('https://www.charitiesnys.com/RegistrySearch/search_charities.jsp')

#identify xpath of location to select element
inputElement = browser.find_element_by_xpath("/html/body/div/div[2]/div/table/tbody/tr/td[2]/div/div/font/font/font/font/font/font/table/tbody/tr[4]/td/form/table/tbody/tr[2]/td[2]/input[1]")
inputElement.send_keys('0')
inputElement1 = browser.find_element_by_xpath("/html/body/div/div[2]/div/table/tbody/tr/td[2]/div/div/font/font/font/font/font/font/table/tbody/tr[4]/td/form/table/tbody/tr[10]/td/input[1]").click()

#identify the table to scrape
table = browser.find_element_by_css_selector('table.Bordered')

print(table)


<selenium.webdriver.remote.webelement.WebElement (session="6f925017b4f0a7937c8e476090e6aa8b", element="f.C2F7697509D3519E5E8EAB1A4BD6AE55.d.3CB50132B001301072C88029315677D7.e.1361")>


In [47]:
# extract data from first page
ls = []
for row in table.find_elements_by_css_selector('tr'):
    cols = ls.append([cell.text for cell in row.find_elements_by_css_selector('td')])

# click page buttoms from 2 to 7 (id: 1-6)
for i in range(1,7):
    page_id = f"/html/body/div[2]/div/table/tbody/tr/td[3]/div/div/span[2]/a[{i}]"
    page_click = browser.find_element_by_xpath(page_id).click()
    table = browser.find_element_by_css_selector('table.Bordered')
    # compile all data
    for row in table.find_elements_by_css_selector('tr'):
        cols = ls.append([cell.text for cell in row.find_elements_by_css_selector('td')])

display(ls)



[[],
 ['"Forever Captain Poodaman" The Ahmad Butler Foundation',
  '48-07-16',
  '843800926',
  'NFP',
  'PHILADELPHIA',
  'PA'],
 ['"Incredibly Blessed" Inc',
  '49-54-61',
  '842071758',
  'NFP',
  'STATEN ISLAND',
  'NY'],
 ['"R" S.U.C.C.E.S.S. Foundation Inc.',
  '49-06-59',
  '874012670',
  'NFP',
  'ROCHESTER',
  'NY'],
 ['"Studio 5404" Inc.', '44-39-58', '463180470', 'NFP', 'MASSAPAQUA', 'NY'],
 ['"THEY ARE HAITIAN" FUND, INC.',
  '20-63-46',
  '300170128',
  'NFP',
  'HUDSON',
  'NY'],
 ['"Y" Dive, Inc.', '48-45-01', '854252095', 'NFP', 'SAINT ALBANS', 'NY'],
 ['(ASMA) American Syrian Multicultural Association, Inc.',
  '42-84-63',
  '273130182',
  'NFP',
  'BROOKLYN',
  'NY'],
 ['#FeedHamburg', '48-37-35', '854150318', 'NFP', 'HAMBURG', 'NY'],
 ['#HicksStrong Inc.', '48-10-48', '842612081', 'NFP', 'CLIFTON PARK', 'NY'],
 ['#WalkAway Foundation', '47-15-80', '832820906', 'NFP', 'CARLSBAD', 'CA'],
 ['04/11 10:17 PM test', '47-13-95', '206256427', 'NFP', 'ALBANY', 'NY'],
 ['1/20/

In [54]:
#####CREATE DATE FRAME#####

df = ls.copy()
#update dataframe with header 
df = pd.DataFrame(df, columns = ["Organization Name", "NY Reg #", "EIN" ,"Registrant Type","City","State"])

#remove blank rows
df.dropna(how='all', ignore_index=True, inplace=True)

display(df) #let's have a look at the data before creating the CSV file and loading it into s3 

,Organization Name,NY Reg #,EIN,Registrant Type,City,State
0,"""Forever Captain Poodaman"" The Ahmad Butler Fo...",48-07-16,843800926,NFP,PHILADELPHIA,PA
1,"""Incredibly Blessed"" Inc",49-54-61,842071758,NFP,STATEN ISLAND,NY
2,"""R"" S.U.C.C.E.S.S. Foundation Inc.",49-06-59,874012670,NFP,ROCHESTER,NY
3,"""Studio 5404"" Inc.",44-39-58,463180470,NFP,MASSAPAQUA,NY
4,"""THEY ARE HAITIAN"" FUND, INC.",20-63-46,300170128,NFP,HUDSON,NY
...,...,...,...,...,...,...
100,"I Love Math, Inc.",41-29-81,262609655,NFP,BRONX,NY
101,"INTERNATIONAL ORGANIZATION FOR ADOLESCENTS, INC.",20-98-81,134093883,NFP,CHICAGO,IL
102,ISAIAH J. JONES MEMORIAL FOUNDATION,20-36-56,760736140,NFP,BRONX,NY
103,Jean Bertrand Foundation For The Disadvantaged...,42-80-65,800735536,NFP,BROOKLYN,NY


In [64]:
###LOAD THE FILE INTO S3####
# prepare csv file name   
pathname = 's3://database-update-bucket2/m10/'#specify location of s3://{my-bucket}/
filename= 'charities_bureau_scrape_' #name of your group
datetime = time.strftime("%Y%m%d%H%M%S") #timestamp
filenames3 = "%s%s%s.csv"%(pathname,filename,datetime) #name of the filepath and csv file

#load file into s3. Pandas actually leverages boto to connect to s3 and can push the file directly into an s3 bucket
df.to_csv(filenames3, header=True) # line_terminator='\n'
# also load file to local
df.to_csv('charities_bureau_scrape.csv')

#print success message
print("Successfull uploaded file to location:"+str(filenames3))

Successfull uploaded file to location:s3://database-update-bucket2/m10/charities_bureau_scrape_20240408173516.csv
